# Vectorized version of the slide 3.4 model

 - It is using the I inelastic selector to iterate
   during the return mapping over all the material
   points simultaneously
 - The gradients of potential functions deliver
   matrices with zero elements - `lambdify` delivers
   inhomogeneous arrays then. Quick fix was using the
   ZERO and ONE variables to include dummy arrays.
   But this produces many zeros for all material points.
   Probably, this indicates the limit of the concept.
   However, for prototyping this is still fine.
 - Would it be possible to use `theano`? It has operators
   for gradients and can run on Tensors. Check it here:
   [theano](https://theano-pymc.readthedocs.io/en/latest/tutorial)


In [1]:
%matplotlib widget
from bmcs_matmod.slide.vslide_34 import Slide34
import bmcs_matmod.slide.vslide_34 as slide_34
import numpy as np

In [2]:
material_params = dict(
    E_T=1, gamma_T=5, K_T=5, S_T=0.6, c_T=1, bartau=1,
    E_N=1, S_N=0.6, c_N = 1, m = 0.01, f_t=1, f_c=20, f_c0=10, eta=0
)

# Verify Slide34

In [3]:
slide = Slide34(**material_params)

To extract the names of the sympy symbols that
can be lambdified or used in a code generation to produce
C code a subclass Cymbol of sp.Symbol is defined
that adds another parameter called codename to be used
in lambdification. Any general material model
contains the property/attribute `state_var_shape`
specifying the key-value pair of the state variable
name and its shape to be allocated.

In [4]:
slide.state_var_shapes

{'s_pi_x': (),
 's_pi_y': (),
 'w_pi': (),
 'z': (),
 'alpha_x': (),
 'alpha_y': (),
 'omega_T': (),
 'omega_N': (),
 'tau_pi_x': (),
 'tau_pi_y': (),
 'sig_pi': (),
 'Z': (),
 'X_x': (),
 'X_y': (),
 'Y_T': (),
 'Y_N': ()}

In [5]:
discr_shape = (1,)

state = {
    var_name: np.zeros(discr_shape + var_shape, dtype=np.float_)
    for var_name, var_shape in
    slide.state_var_shapes.items()
}

In [6]:
state

{'s_pi_x': array([0.]),
 's_pi_y': array([0.]),
 'w_pi': array([0.]),
 'z': array([0.]),
 'alpha_x': array([0.]),
 'alpha_y': array([0.]),
 'omega_T': array([0.]),
 'omega_N': array([0.]),
 'tau_pi_x': array([0.]),
 'tau_pi_y': array([0.]),
 'sig_pi': array([0.]),
 'Z': array([0.]),
 'X_x': array([0.]),
 'X_y': array([0.]),
 'Y_T': array([0.]),
 'Y_N': array([0.])}

In [7]:
eps_Ema = np.zeros(discr_shape+(3,), dtype=np.float_)

In [8]:
s_x_n1, s_y_n1, w_n1 = np.einsum('...a->a...',eps_Ema)

Eps_n = np.array([state[eps_name] for eps_name in slide.Eps_names])
Eps_k = np.copy(Eps_n)
Sig_k = np.array([state[sig_name] for sig_name in slide.Sig_names])

In [9]:
s_x_n1.shape

(1,)

In [10]:
Eps_k.shape
Sig_k.shape

(8, 1)

In [11]:
slide.symb.get_Sig_(s_x_n1, s_y_n1, w_n1, Sig_k, Eps_k)[0].shape

(8, 1)

In [12]:
slide.symb.Sig_

Matrix([[-E_T*(1 - omega_T)*(s^{\pi}_x - s_x), -E_T*(1 - omega_T)*(s^{\pi}_y - s_y), -E_N*(-2*w + 2*w^{\pi})*(-omega_N*Piecewise((0, \sigma^\pi <= 0), (1, True))/2 + 1/2), K_T*z, alpha_x*gamma_T, alpha_y*gamma_T, E_T*(-s^{\pi}_x + s_x)**2/2 + E_T*(-s^{\pi}_y + s_y)**2/2, E_N*(w - w^{\pi})**2*Piecewise((0, \sigma^\pi <= 0), (1, True))/2]])

In [13]:
slide.symb.dSig_dEps_

Matrix([
[    E_T*I*(omega_T - 1),                     I*O,                                                             I*O,   I*O,       I*O,       I*O, E_T*I*(s^{\pi}_x - s_x),                                                                   I*O],
[                    I*O,     E_T*I*(omega_T - 1),                                                             I*O,   I*O,       I*O,       I*O, E_T*I*(s^{\pi}_y - s_y),                                                                   I*O],
[                    I*O,                     I*O,  E_N*I*(omega_N*Piecewise((O, O >= \sigma^\pi), (1, True)) - 1),   I*O,       I*O,       I*O,                     I*O, E_N*I*(-2*w + 2*w^{\pi})*Piecewise((O, O >= \sigma^\pi), (1, True))/2],
[                    I*O,                     I*O,                                                             I*O, I*K_T,       I*O,       I*O,                     I*O,                                                                   I*O],
[                    I*

In [14]:
ONES = np.ones_like(s_x_n1)
ZEROS = np.zeros_like(s_x_n1)

In [15]:
dSig_dEps_k = slide.symb.get_dSig_dEps_(s_x_n1, s_y_n1, w_n1, Sig_k, Eps_k, ZEROS, ONES)
dSig_dEps_k.shape

(8, 8, 1)

In [16]:
ix1, ix2 = np.ix_((0,1,2),(0,1,2))

In [17]:
dSig_dEps_k[ix1,ix2,0]

array([[-1.,  0.,  0.],
       [ 0., -1.,  0.],
       [ 0.,  0., -1.]])

In [18]:
H_sig_pi = slide.symb.get_H_sig_pi_(Sig_k)
H_sig_pi.shape

(1,)

In [19]:
f_k = slide.symb.get_f_(Eps_k, Sig_k, H_sig_pi)
f_k.shape

(1,)

In [20]:
%%capture
df_dSig_k = slide.symb.get_df_dSig_(Eps_k, Sig_k, H_sig_pi, ZEROS, ONES)

In [21]:
df_dSig_k.shape

(8, 1, 1)

In [22]:
%%capture
ddf_dEps_k = slide.symb.get_ddf_dEps_(Eps_k, Sig_k, H_sig_pi, ZEROS, ONES)

In [23]:
ddf_dEps_k.shape

(8, 1, 1)

In [24]:
df_dEps_k = np.einsum(
    'ik...,ji...->jk...', df_dSig_k, dSig_dEps_k) + ddf_dEps_k
df_dEps_k.shape

(8, 1, 1)

In [25]:
%%capture
slide.get_f_df(s_x_n1, s_y_n1, w_n1, Sig_k, Eps_k)

In [26]:
slide.get_Eps_k1(s_x_n1, s_y_n1, w_n1, Eps_n, ONES, Sig_k, Eps_k)

array([[0.  ],
       [0.  ],
       [0.99],
       [1.  ],
       [0.  ],
       [0.  ],
       [0.  ],
       [0.  ]])

In [36]:
discr_shape = (2,1)

state = {
    var_name: np.zeros(discr_shape + var_shape, dtype=np.float_)
    for var_name, var_shape in
    slide.state_var_shapes.items()
}
eps_Ema = np.zeros(discr_shape+(3,), dtype=np.float_)
eps_Ema.shape

(2, 1, 3)

In [37]:
s_max = 3
eps_Ema[0,0,0] = s_max
slide.k_max = 3
sig_Ema = slide.get_corr_pred(eps_Ema, 1, **state)

In [38]:
sig_Ema

(array([[[-1.02475905,  0.        , -0.00181817]],
 
        [[-0.        , -0.        , -0.        ]]]),
 array([[[[1., 0., 0.],
          [0., 1., 0.],
          [0., 0., 1.]]],
 
 
        [[[1., 0., 0.],
          [0., 1., 0.],
          [0., 0., 1.]]]]))

In [24]:
from bmcs_matmod.slide.slide_32 import Slide32
Sig_k_32 = np.zeros((8,))
Eps_k_32 = np.zeros((8,))
slide32 = Slide32(**material_params)
slide32.get_sig_n1(s_max,0,0,Sig_k_32, Eps_k_32,3)

(array([0.18181653, 0.        , 0.00181817, 0.18181653, 0.18181653,
        0.        , 1.36362397, 0.        ]),
 array([-1.02475905e+00,  0.00000000e+00, -1.81816529e-03,  9.09082645e-01,
         9.09082645e-01,  0.00000000e+00,  3.97107904e+00,  0.00000000e+00]),
 2)